<a href="https://colab.research.google.com/github/zahraDehghanian97/Brain_cancer_classification/blob/master/preprocessing_mat_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import h5py
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from PIL import Image


In [2]:
#Connect Google Drive to Colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#Drive REST API and authenticate

from google.colab import auth
auth.authenticate_user()

In [12]:
# change dir
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')
os.mkdir("Brain Cancer")
!ls -a

 4_link_predict.ipynb
'Algorithm2_for_counting_Triangles (3).ipynb'
 ARIMA.ipynb
'base_models (1).ipynb'
 base_models.ipynb
 bda_hw1_2.ipynb
 BiLSTM_POS_Tagging.ipynb
 Classification.ipynb
 colab_pro_2.ipynb
'Copy of PU_Biased_CIFAR10(1).ipynb'
'Copy of simpleALAD.ipynb'
 CS246_1aPy2.ipynb
 CS246_2_1.ipynb
 CS246_2_2.ipynb
'dgl_gnn_large (1).ipynb'
'dgl_gnn_large (2).ipynb'
 dgl_gnn_large.ipynb
'ENGNN (4).ipynb'
 ENGNN_run_1.ipynb
 ENGNN_run_2.ipynb
 ENGNN_run.ipynb
 gc_const_new.ipynb
 gnlp_prs.ipynb
 gnn_extend_1.ipynb
'gnn_extend(1).ipynb'
 GNN-extend.ipynb
 gnn_extend_new_names.ipynb
 gnn_extend-new-names.ipynb
 graph_construction_dep_seqlink_two.ipynb
 hw1_3.ipynb
'IRSE_Project(1).ipynb'
'last (1).ipynb'
 last2.ipynb
 last.ipynb
'last_simple_(1).ipynb'
 link_predict_final.ipynb
 link_predict.ipynb
 link_predict_new.ipynb
 ML_project_1.ipynb
 nlp_hw1_final_1.ipynb
 nlp_hw1.ipynb
'NLP_PART2_(1).ipynb'
 node_classification_final_experiments_1.ipynb
 node_classification_final_experime

In [14]:
#Download Files for google drive
from google.colab import files
import requests 
file_url = "https://ndownloader.figshare.com/articles/1512427/versions/5"
	
r = requests.get(file_url, stream = True) 

with open("/content/drive/My Drive/Colab Notebooks/Brain Cancer/data.zip", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

In [27]:
#unzip data in dataset
!unzip -q "/content/drive/MyDrive/Colab Notebooks/Brain Cancer/data.zip" -d "/content/drive/MyDrive/Colab Notebooks/Brain Cancer/dataset/"


In [29]:
##Extract All zip into Mat Files
import os, zipfile

dir_name = '/content/drive/My Drive/Colab Notebooks/Brain Cancer/dataset'
Extract_dir_name1 = '/content/drive/My Drive/Colab Notebooks/Brain Cancer/Mat Files'
os.chdir('/content/drive/My Drive/Colab Notebooks/Brain Cancer/dataset')

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(".zip"): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        print (file_name) #Print zip file Name
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(Extract_dir_name1) # extract file to Extract dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

     

/content/drive/My Drive/Colab Notebooks/Brain Cancer/dataset/brainTumorDataPublic_1-766.zip
/content/drive/My Drive/Colab Notebooks/Brain Cancer/dataset/brainTumorDataPublic_1533-2298.zip
/content/drive/My Drive/Colab Notebooks/Brain Cancer/dataset/brainTumorDataPublic_767-1532.zip
/content/drive/My Drive/Colab Notebooks/Brain Cancer/dataset/brainTumorDataPublic_2299-3064.zip


In [30]:
def mat_file_to_dict(filepath: str) -> dict:
    tumor_class = {1: 'meningioma', 2: 'glioma', 3: 'pituitary_tumor'}
    tumor_data_dict = {}
    with h5py.File(filepath, mode = 'r') as image_data:
        cjdata_struct = image_data['cjdata']
        tumor_data_dict['class'] = tumor_class[int(cjdata_struct['label'][0, 0])]
        tumor_data_dict['image'] = cjdata_struct['image'][:].transpose()
        tumor_data_dict['tumor_border'] = cjdata_struct['tumorBorder'][0]
        tumor_data_dict['tumor_mask'] = cjdata_struct['tumorMask'][:].transpose()
    return tumor_data_dict

In [31]:
def generate_img_from_mat(mat_file_data: dict) -> None:
    plt.figure(figsize = (15, 7))
    tumor_mri = plt.imshow(mat_file_data['image'], cmap = 'gray')
    plt.axis('off')

In [ ]:
rootpath_dataset = '/content/drive/My Drive/Colab Notebooks/Brain Cancer'
mat_dirname = 'Mat Files'
training_dir = os.path.join(rootpath_dataset, 'Training')

m_count, g_count, p_count = (0, 0, 0)
for root, dirpath, filenames in os.walk(os.path.join(rootpath_dataset, mat_dirname)):
    for curr_file in filenames:
        mri_img_data = mat_file_to_dict(os.path.join(root, curr_file))
        tumor_class = mri_img_data['class']
        generate_img_from_mat(mri_img_data)
        if tumor_class == 'meningioma':
            m_count += 1
            m_path = os.path.join(training_dir, 'meningioma')
            if not os.path.exists(m_path):
                os.mkdir(m_path)
            plt.savefig(os.path.join(m_path, f'M{m_count}.jpg'), dpi = 300, bbox_inches = 'tight', pad_inches = 0)
        elif tumor_class == 'glioma':
            g_count += 1
            g_path = os.path.join(training_dir, 'glioma')
            if not os.path.exists(g_path):
                os.mkdir(g_path)
            plt.savefig(os.path.join(g_path, f'G{g_count}.jpg'), dpi = 300, bbox_inches = 'tight', pad_inches = 0)
        elif tumor_class == 'pituitary_tumor':
            p_count += 1
            p_path = os.path.join(training_dir, 'pituitary_tumor')
            if not os.path.exists(p_path):
                os.mkdir(p_path)
            plt.savefig(os.path.join(p_path, f'P{p_count}.jpg'), dpi = 300, bbox_inches = 'tight', pad_inches = 0)
        else:
            pass

<ipython-input-31-60754b8307cf>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize = (15, 7))
